```c
GET http://127.0.0.1:8080/greenhouse_1/temperature?num=10&date=20220828

Method: GET
Root Endpoint: http://172.0.0.1:8080
Path: /greenhouse_1/temperature
Query Parameters: num=10&date=20220828
```

关于设计rest api, 我有话想说. 上面例子`/greenhouse_1/temperature`是path, 属于endpoint的一部分, 在写Routing代码的时候, 只应考虑endpoint范围, 不应该把query parameters加进去. 

```js
// 这么设计endpoint是错误的, 不应该把query参数加进去
app.get('/humidity/date/:date/num/:num', (req, res) => {
    res.send('date=' + `${req.params.date}` + 'num=' + `${req.params.num}`);
});

// 正确如下, 即 用req.query来访问query parameters
app.get('/humidity', (req, res) => {
    res.send('data from humidity');
    console.log(Object.keys(req.query).length);
    console.log(req.query.date);
    console.log(req.query.num);
});
```

express官网对Routing的定义如下(通过定义也可以看出, 设计endpoint的时候并不能把query parameters考虑进去):

> Routing refers to how an application’s `endpoints (URIs)` respond to client requests. 

设计rest api的时候应该考虑, 哪些东西应该放到endpoint(logical nesting), 哪些应该是query parameter(non sensitive), 而哪些应该放到headers(sensitive, password), 哪些应该是body里面的东西. 

# 如何测试
---

设计完成后可以通过curl或者在浏览器输入地址来测试. 

命令行输入如下:
```shell
curl -X GET "http://127.0.0.1:3000/humidity?date=3&num=6"
```

在浏览器输入:

`http://127.0.0.1:3000/humidity?date=3&num=6`

# Accept and respond with JSON
---

The POST, PUT, and DELETE endpoints all take JSON as the request body, and they all return JSON as the response, including the GET endpoint. 意思就是说, 作为rest api, 对于body的要求应该是json, 不是说必须, 而是说这样比较好. 比如你返回一个body的时候, 应该在headers里加一个`Content-Type: application/json`.

# Use nouns instead of verbs in `endpoint paths`
---

path使用名词来定义, 而不是动词. 

```js
app.get('/articles', (req, res) => {
  const articles = [];
  // code to retrieve an article...
  res.json(articles);
});

app.delete('/articles/:id', (req, res) => {
  const { id } = req.params;
  // code to delete an article...
  res.json({ deleted: id });
});
```
In the code above, we defined the endpoints to manipulate articles. As we can see, the path names do not have any verbs in them. All we have are nouns. The verbs are in the HTTP verbs.

# Use logical nesting on endpoints
---

For example, if we want an endpoint to get the comments for a news article, we should append the /comments path to the end of the /articles path. We can do that with the following code in Express:

```js
const express = require('express');
const bodyParser = require('body-parser');

const app = express();

app.use(bodyParser.json());

app.get('/articles/:articleId/comments', (req, res) => {
  const { articleId } = req.params;
  const comments = [];
  // code to get comments by articleId
  res.json(comments);
});


app.listen(3000, () => console.log('server started'));
```

In the code above, we can use the GET method on the path `'/articles/:articleId/comments'`. We get comments on the article identified by articleId and then return it in the response. We add `'comments'` after the `'/articles/:articleId'` path segment to indicate that it’s a child resource of `/articles`.

This makes sense since `comments` are the children objects of the `articles`, assuming each article has its own comments. Otherwise, it’s confusing to the user since this structure is generally accepted to be for accessing child objects. The same principle also applies to the POST, PUT, and DELETE endpoints. They can all use the same kind of nesting structure for the path names.


# Handle errors gracefully and return standard error codes
---

Common error HTTP status codes include:

400 Bad Request – This means that client-side input fails validation.

401 Unauthorized – This means the user isn’t not authorized to access a resource. It usually returns when the user isn’t authenticated.

403 Forbidden – This means the user is authenticated, but it’s not allowed to access a resource.

404 Not Found – This indicates that a resource is not found.

500 Internal server error – This is a generic server error. It probably shouldn’t be thrown explicitly.

502 Bad Gateway – This indicates an invalid response from an upstream server.

503 Service Unavailable – This indicates that something unexpected happened on server side (It can be anything like server overload, some parts of the system failed, etc.).

We should be throwing errors that correspond to the problem that our app has encountered. For example, if we want to reject the data from the request payload, then we should return a 400 response as follows in an Express API:

```js
const express = require('express');
const bodyParser = require('body-parser');

const app = express();

// existing users
const users = [
  { email: 'abc@foo.com' }
]

app.use(bodyParser.json());

app.post('/users', (req, res) => {
  const { email } = req.body;
  const userExists = users.find(u => u.email === email);
  if (userExists) {
    return res.status(400).json({ error: 'User already exists' })
  }
  res.json(req.body);
});


app.listen(3000, () => console.log('server started'));
```

# Allow filtering, sorting, and pagination
---

```js
const express = require('express');
const bodyParser = require('body-parser');

const app = express();

// employees data in a database
const employees = [
  { firstName: 'Jane', lastName: 'Smith', age: 20 },
  //...
  { firstName: 'John', lastName: 'Smith', age: 30 },
  { firstName: 'Mary', lastName: 'Green', age: 50 },
]

app.use(bodyParser.json());

app.get('/employees', (req, res) => {
  const { firstName, lastName, age } = req.query;
  let results = [...employees];
  if (firstName) {
    results = results.filter(r => r.firstName === firstName);
  }

  if (lastName) {
    results = results.filter(r => r.lastName === lastName);
  }

  if (age) {
    results = results.filter(r => +r.age === +age);
  }
  res.json(results);
});

app.listen(3000, () => console.log('server started'));
```

In the code above, we have the `req.query` variable to get the query parameters. We then extract the property values by destructuring the individual query parameters into variables using the JavaScript destructuring syntax. Finally, we run `filter` on with each query parameter value to locate the items that we want to return.

Once we have done that, we return the results as the response. Therefore, when we make a GET request to the following path with the query string:

`/employees?lastName=Smith&age=30`

We get:

```json
[
    {
        "firstName": "John",
        "lastName": "Smith",
        "age": 30
    }
]
```

as the returned response since we filtered by lastName and age.

Likewise, we can accept the page query parameter and return a group of entries in the position from (page - 1) * 20 to page * 20. 

We can also specify the fields to sort by in the query string. For instance, we can get the parameter from a query string with the fields we want to sort the data for. Then we can sort them by those individual fields.

For instance, we may want to extract the query string from a URL like:

`http://example.com/articles?sort=+author,-datepublished`

Where `+` means ascending and `-` means descending. So we sort by author’s name in alphabetical order and datepublished from most recent to least recent.

了解更多设计相关的规范:

https://stackoverflow.blog/2020/03/02/best-practices-for-rest-api-design/